In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch/minsearch.py

--2025-06-16 22:03:28--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5820 (5.7K) [text/plain]
Saving to: ‘minsearch.py.4’

minsearch.py.4      100%[===================>]   5.68K  --.-KB/s    in 0s      

2025-06-16 22:03:28 (68.2 MB/s) - ‘minsearch.py.4’ saved [5820/5820]



In [2]:
import minsearch

In [3]:
import json

In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [8]:
q = 'the course has already started, can I still join?'

In [9]:
index.fit(documents)

In [10]:
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query=q,
    filter_dict={"course": "data-engineering-zoomcamp"},
    boost_dict=boost,
    num_results=5
)

In [11]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineerin

In [12]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [13]:
import groq

In [14]:
from groq import Client

In [15]:
client = Client(api_key=os.getenv("GROQ_API_KEY"))

In [16]:
q

'the course has already started, can I still join?'

In [17]:
response = client.chat.completions.create(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    messages=[
      {
        "role": "user",
        "content": q
      }
]
)

response.choices[0].message.content

'The answer depends on various factors, which I\'ll outline below. Please consider these points, and if you\'re still unsure, feel free to provide more context or details about the course.\n\n**Factors to consider:**\n\n1. **Course type**: Is it an online or offline course? Online courses might be more flexible with enrollment, while offline courses might have more rigid structures.\n2. **Course format**: Is it a synchronous (live) or asynchronous (pre-recorded) course? Synchronous courses might have already started with a specific cohort, while asynchronous courses might allow you to join at any time.\n3. **Instructor\'s policy**: The instructor or course administrator might have a specific policy regarding late enrollment. Some might allow it, while others might not.\n4. **Course structure**: Are there prerequisites or a specific sequence of lessons? If so, joining mid-course might be more challenging.\n\n**Possible scenarios:**\n\n* **Online courses with flexible enrollment**: You m

In [18]:
prompt_template = """

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.


QUESTION: {question}

CONTEXT: 
{context}

""".strip()

In [19]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [20]:
prompt = prompt_template.format(question=q, context=context).strip()

In [21]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.


QUESTION: the course has already started, can I still join?

CONTEXT: 
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1
Subscribe to course public Google Calendar (it works from Desktop only).
Register before the course starts using this link.
J

In [22]:
response = client.chat.completions.create(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    messages=[
      {
        "role": "user",
        "content": prompt
      }
]
)

response.choices[0].message.content

"According to the CONTEXT, the answer to your QUESTION is:\n\nYes, even if you don't register, you're still eligible to submit the homeworks. However, be aware that there will be deadlines for turning in the final projects, so don't leave everything for the last minute."

In [23]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
    query=query,
    filter_dict={"course": "data-engineering-zoomcamp"},
    boost_dict=boost,
    num_results=5
    )
    
    return results

In [28]:
def build_prompt(query, search_results):
    prompt_template = """

    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    

    QUESTION: {question}

    CONTEXT: 
    {context}

    """.strip()
    
    
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
        
    prompt = prompt_template.format(question=query, context=context).strip()
    
    return prompt

In [29]:
def llm(prompt):
        response = client.chat.completions.create(
            model="meta-llama/llama-4-scout-17b-16e-instruct",
            messages=[
              {
                "role": "user",
                "content": prompt
              }
        ]
        )

        return response.choices[0].message.content

In [32]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    
    return answer

In [34]:
rag(query)

"To run Kafka, you need to consider the type of program you're trying to execute, as instructions vary between Java and Python implementations.\n\n- **For Java Kafka Producer/Consumer/KStreams:** Navigate to your project directory and execute the command:\n  ```\n  java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n  ```\n  Replace `<jar_name>` with the actual name of your JAR file.\n\n- **For Python Kafka Producer (if using a Python environment):** \n  1. First, ensure you have a virtual environment. If not, create one and activate it:\n     ```\n     python -m venv env\n     source env/bin/activate\n     ```\n  2. Install required packages:\n     ```\n     pip install -r ../requirements.txt\n     ```\n  3. Run your Python file within this activated environment.\n\n- **Troubleshooting:**\n  - If you encounter a `./build.sh: Permission denied Error`, run:\n    ```\n    chmod +x build.sh\n    ```\n  - If you face a `ModuleNotFoundError: No m

In [35]:
rag('the course has already started, can I still enroll?')

"According to the CONTEXT, the answer to your QUESTION is:\n\nYes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

In [39]:
print(rag("if I don't submit the assignment, will I still get certificate?"))

Based on the provided CONTEXT, there is no direct information about what happens to certificate eligibility if an assignment is not submitted. However, it does mention that to get a certificate, you need to finish the course with a “live” cohort and that peer-reviewing capstones after submitting a project is required.

Given this, it can be inferred that not submitting an assignment (and thus not being able to peer-review others' projects) would likely impact your ability to receive a certificate, but there's no explicit statement in the CONTEXT that directly answers the QUESTION.

The closest information we have is: 

"No, you can only get a certificate if you finish the course with a “live” cohort. We don't award certificates for the self-paced mode. The reason is you need to peer-review capstone(s) after submitting a project."

So, based on this, not submitting the assignment might hinder the process of getting a certificate since submitting a project is a prerequisite for peer-revi